# Langgraph
`07_langgraph_intro.ipynb`

**학습자료**
- Langchain academy
- Officoal Documentation
- Teddy Note

## Langgraph
- State(상태) 기반 워크플로우 오케스트레이션 프레임워크
- 그래프 구조: Node(정점) / Edge(간선) 의 형태로 설계 & 실행 가능
- 조건 분기: 특정 노드의 결과에 따라 다음 경로가 달라짐


In [ ]:
%pip install -q langgraph

In [35]:
# Typed Dict 방식 -> 가벼움(힌트)
from typing_extensions import TypedDict

class State(TypedDict):
    graph_state: str
    history: list

'''
# Pydantic 방식 -> 무거운(검증)
from pydantic import BaseModel

class State(BaseModel):
    graph_state: str
'''

# 걍 딕셔너리인데, 키값에 대한 사전정의 + value 의 타입을 설정함(강제X 힌트O)
s1 = State()
s1['graph_state'] = 'asdf'
s1

{'graph_state': 'asdf'}

In [ ]:
# Node 정의 (함수 - state 를 parameter로 받음)

def node_1(state: State):
    new_str = state['graph_state'] + ' I am '
    state['history'].append('node1')
    return {'graph_state': new_str}


def node_2(state: State):
    state['history'].append('node2')
    new_str = state['graph_state'] + 'Happy!!'
    return {'graph_state': new_str}


def node_3(state: State):
    state['history'].append('node3')
    new_str = state['graph_state'] + 'Sad!!'
    return {'graph_state': new_str}

In [29]:
# 조건 분기(Conditional Edge)

def decide_mood(state):
    user_input = state['graph_state']
    # 사용자 입력의 글자수가 홀수면 node2 로 보내고
    if len(user_input) % 2:
        return '노드2'
    # 사용자 입력이 짝수면 node3 로 보낸다.
    else:
        return '노드3'

In [30]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END

builder = StateGraph(State)
# 노드 등록은 k-v 형식
builder.add_node('노드1', node_1)
builder.add_node('노드2', node_2)
builder.add_node('노드3', node_3)

# 연결 (Edge)
builder.add_edge(START, '노드1')
builder.add_conditional_edges('노드1', decide_mood)
builder.add_edge('노드2', END)
builder.add_edge('노드3', END)

# 그래프 빌드
graph = builder.compile()

In [ ]:
display(Image(graph.get_graph().draw_mermaid_png()))

In [34]:
graph.invoke({'graph_state': '안녕하'})

---- Node1 ----
---- Node2 ----


{'graph_state': '안녕하 I am Happy!!'}